In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import math

In [ ]:
def check_numeric(X):
  newX = np.array(X).reshape(-1)
  return all(not isinstance(n, str) for n in newX)

# X=np.array([[6,7,8],[3,0,5]])
# print(check_numeric(X))
# print(X)
# print(check_numeric([1,2,3]))

In [ ]:
def error_return(p):
  ranked = np.arange(p, dtype=int) 
  weight = np.empty((1,p))
  weight = np.squeeze(weight) 
  weight[:] = np.nan
  return ranked, weight


### Preprocessing data and call feature selection algorithm

In [ ]:
def Relief_configure(X,Y, **kwargs):
  X = np.array(X)
  Y = np.array(Y)
  numUpdates, categoricalX = kwargs['numUpdates'], kwargs['categoricalX']
  K = 1 # number of nearest neighbours

  # if not check_numeric(X):
  #   print('X does not contain numeric data')
  #   p = X.shape[1] # no of attributes
  #   return error_return(p) 

  # Check if the input sizes are consistent
  if Y.shape[0] != X.shape[0]:
    print('number of instances and output labels doesnot match')
    p = X.shape[1] # no of attributes
    return error_return(p)
  
  # converting classes as 0 to #classes
  [Y, grp] = pd.factorize(Y)

  # grpToInd contains class to index mapping, grpToInd[className]= ind
  grpToInd={}
  for ind, g in enumerate(grp):
    grpToInd[g]= ind

  # removing incomplete instances
  # df_XY = pd.DataFrame(X)
  # df_XY['Y'] = Y
  # df_XY = df_XY.dropna()
  # X, Y = np.array(df_XY.iloc[:, 0:-1]), np.array(df_XY.iloc[:, -1])

  Ngrp = len(grp)
  N = X.shape[0]
  C = np.zeros((N,Ngrp))
  C[np.arange(N), Y] = True
  
  # Checking the number of classes
  if Ngrp>2:
    print('Number of classes exceed 2')
    p = X.shape[1] # no of attributes
    return error_return(p)

  #  Do we have enough observations?
  if len(Y)<2:
    print('not enough instances')
    p = X.shape[1] # no of attributes
    return error_return(p)

  # Check number of updates
  if (not numUpdates=='all') and (not check_numeric([numUpdates]) or numUpdates<=0):
    print('numUpdates is invalid')
  elif (not numUpdates) or numUpdates=='all':
    numUpdates = X.shape[0]
  else:
    numUpdates = math.ceil(numUpdates)
    
  # Check the type of categoricalX
  if not categoricalX or (categoricalX != 'on' and categoricalX != 'off'):
      print('categoricalX is invalid')
  categoricalX = (categoricalX == 'on')   

  # Find max and min for every predictor
  p = X.shape[1] # no of attributes
  Xmax = X.max(0)
  Xmin = X.min(0) 
  Xdiff = Xmax - Xmin
  Xmean = np.mean(X, axis=0) 

  # Exclude single-valued attributes
  isDiffValue = Xdiff >= 1e-9  # boolean array of size #attributes [1,0,0,0]
  if not any(isDiffValue):
    print("All attributes are single valued attributes.")
    p = X.shape[1] # no of attributes
    return error_return(p)
  
  X = X[:, isDiffValue ] 
  Xmax = Xmax[isDiffValue] 
  Xmin = Xmin[isDiffValue]
  Xdiff = Xdiff[isDiffValue] 
  Xmean = Xmean[isDiffValue] 
  rejected = [ i for i in range(len(isDiffValue)) if not isDiffValue[i]]  # indices of the deleted attributes (values range from 1 to p)
  accepted = [ i for i in range(len(isDiffValue)) if isDiffValue[i]]  # indices of remaining attributes (values range from 1 to p)

  # Scale and center the attributes
  if not categoricalX:
      X = (X - Xmin) / Xdiff 

  # The #updates cannot be more than the #observations
  numUpdates = min(numUpdates, X.shape[0])

  # Call Relief. By default all weights are set to NaN.
  weight = np.empty(p) 
  weight[:] = np.nan

  weight[accepted] = Relief(X,C,numUpdates,K,categoricalX) 

  # Assign ranks to attributes
  sorted = np.argsort(-weight[accepted])
  accepted = np.array(accepted)
  ranked = accepted[sorted]
  ranked = np.append(ranked, rejected)
  ranked = ranked.astype(int)

  return ranked, weight



# Relief algorithm function

In [ ]:
def Relief(scaledX,C,numUpdates,K,categoricalX):
  
  # Relief for classification
  numInstances,numAttr = scaledX.shape 
  attrWeights = np.zeros(numAttr) 
  # C is boolean 2D matrix of size (N, Ngrp) i.e. (Number of instances vs Number of classes)
  numClasses = C.shape[1]   # Ngrp

   # Choose the random instances
  np.random.seed(123) 
  rndIdx = np.random.choice(numInstances, numUpdates, replace=False)   # random indices of m instances from total of n instances
  idxVec = np.arange(numInstances)   # [0 .. N-1]

   # Make searcher objects, one object per class. 
  instIdxPerClass = {}  
  for c in range(numClasses):
      c_C = C[:,c]
      instIdxPerClass[c] = np.array([i for i in range(len(c_C)) if c_C[i]])   # instances of class c
  
  # selecting distance function
  distFunc = 'manhattan'
  if categoricalX:
    distFunc = 'hamming'
  else: 
    distFunc = 'manhattan'
  
   # Outer loop, for updating attribute weights iteratively
  for i in range(numUpdates):
    targetIdx = rndIdx[i]
    
      # Choose the correct random observation
    targetInst = scaledX[targetIdx,:]

      # Find the class for this observation
    targetC = C[targetIdx, :]
    thisC = np.array([i for i in range(len(targetC)) if targetC[i]]).item() # taking the class of target instance
    
      # Find the k-nearest hits 
    sameClassIdx = instIdxPerClass[thisC] 
    
      # we may not always find numNeighbor Hits
    lenHits = min(len(sameClassIdx)-1,K) 

      # find nearest hits
      # It is not guaranteed that the first hit is the same as targetIdx. Since
      # they have the same class, it does not matter. If we add observation
      # weights in the future, we will need here something similar to what we
      # do in ReliefReg.
    # print('HitFit : ', scaledX[sameClassIdx])
    Hits = [] 
    if lenHits>0:
        nearestNeighborHits = NearestNeighbors(n_neighbors=lenHits+1, algorithm='auto', metric= distFunc).fit(scaledX[sameClassIdx])
        idxH = nearestNeighborHits.kneighbors([targetInst], lenHits+1, return_distance=False)
        idxH = np.squeeze(idxH)
        Hits = sameClassIdx[idxH[1:]] 

      # Process misses
    missClass = [i for i in range(len(targetC)) if not targetC[i]]
    Misses = [] 
    
    if len(missClass):  # Make sure there are misses!
          # Find the k-nearest misses Misses(C,:) for each class C ~= class(targetInst)
          # Misses will be of size (no. of classes -1)x(K)
        Misses = np.empty((numClasses-1, min(numInstances,K+1)), dtype= int)   # last column has class index
        Misses[:] = -1

        for mi in range(len(missClass)):
            
              # find all observations of this miss class
            missClassIdx = instIdxPerClass[missClass[mi]]
            
              # we may not always find K misses
            lenMiss = min(len(missClassIdx),K) 
            
            # print('Missfit :', scaledX[missClassIdx])

              # find nearest misses
            if lenMiss>0:
              nearestNeighborMiss = NearestNeighbors(n_neighbors=lenMiss, algorithm='auto', metric= distFunc).fit(scaledX[missClassIdx])
              idxM = nearestNeighborMiss.kneighbors([targetInst], lenMiss, return_distance=False)
              idxM = np.squeeze(idxM)
              Misses[mi, 0:lenMiss] = missClassIdx[idxM]
        
          # Misses contains obs indices for miss classes, sorted by dist.
        Misses[:,-1] = missClass 
            
      #***************** ATTRIBUTE UPDATE *****************************
      # Inner loop to update weights for each attribute:

    # print('targetID :',targetIdx)
    # print("target :",targetInst)
    # print('HitId :', Hits)
    # print("Hit :", X[Hits])
    # print('MissId :', Misses[:, 0:-1])
    # print("Miss :", X[Misses[:, 0:-1]])
    
    for j in range(numAttr):
        dH = diffH(j,scaledX,targetIdx,Hits, categoricalX)/numUpdates
        # print('dH', dH) 
        dM = diffM(j,scaledX,targetIdx,Misses,categoricalX)/numUpdates
        # print('dM', dM)
        attrWeights[j] = attrWeights[j] - dH + dM 
      
      #****************************************************************

  return attrWeights    

In [ ]:
def cityblock(thisX, X):
  d = abs(X - thisX) 
  return d

In [ ]:
def hamming(thisX, X):
  d = (X != thisX).astype(int) 
  return d


In [ ]:
#--------------------------------------------------------------------------
# DIFFH (for ReliefClass): Function to calculate difference measure
# for an attribute between the selected instance and its hits

def diffH(a, X, targetIdx, Hits, categoricalX):

  # print(a, targetIdx, Hits, categoricalX)

  distMeas = 0 

  # If no hits, return zero by default
  if not len(Hits):
      return distMeas

  # Calculate weighted sum of distances
  if categoricalX:
    distMeas = np.sum(hamming(X[targetIdx, a], X[Hits,a]))
  else:
    distMeas = np.sum(cityblock(X[targetIdx, a], X[Hits,a]))

  # print('Hit distmeas : ', distMeas/len(Hits))

  return distMeas/len(Hits)


In [ ]:
#--------------------------------------------------------------------------
# DIFFM (for ReliefClass) : Function to calculate difference measure
# for an attribute between the selected instance and its misses
def diffM(a, X, targetIdx, Misses, categoricalX):
  distMeas = 0 
  MissWithoutC = Misses[:, 0:-1]
  # If no misses, return zero
  if not len(Misses):
    return distMeas

  # Loop over misses
  for mi in range(Misses.shape[0]):
    ismiss = (Misses[mi, 0:-1] != -1) 
    
    if any(ismiss):
        cls = Misses[mi,-1] 
        nmiss = np.sum(ismiss)           

        if categoricalX:
          distMeas = distMeas + (np.sum(hamming(X[targetIdx, a], X[MissWithoutC[mi, ismiss], a]))/nmiss)
        else:
          distMeas = distMeas + (np.sum(cityblock(X[targetIdx, a], X[MissWithoutC[mi,ismiss], a]))/nmiss)

  # print('Miss distmeas : ', distMeas)

  return distMeas


In [ ]:
# X = np.array([[0,0,0,0,1,0,0],
#               [0,1,0,1,0,1,0],
#               [1,0,0,0,1,1,0],
#               [0,0,0,1,0,0,0],
#               [1,1,0,0,0,0,0],
#               [0,1,0,0,1,0,0],
#               [0,0,0,0,0,1,0],
#               [1,0,0,1,0,0,0],
#               [1,0,1,0,0,0,1],
#               [1,1,0,0,1,0,1],
#               [0,0,0,0,0,0,1],
#               [1,0,1,1,1,1,1],
#               [1,0,0,0,0,0,0],
#               [1,0,0,1,0,1,1],
#               [0,1,1,0,0,0,0],
#               [0,0,1,1,0,0,1],
#               [1,1,0,1,0,0,0],
#               [1,0,1,0,1,0,1],
#               [0,1,1,1,0,0,1],
#               [1,1,1,1,1,1,1]])

# Y = np.array([0,1,1,0,0,1,0,1,1,0,0,1,1,1,1,0,0,1,1,0])

# Relief_configure(X,Y,K=3,prior='uniform', numUpdates='all', categoricalX='on')


In [ ]:
# X = np.array([[1,0,1],
#               [1,0,0],
#               [0,1,1],
#               [0,1,0],
#               [0,0,1],
#               [0,0,0],
#               [1,1,1],
#               [1,1,0]])

# Y = np.array([1,1,1,1,0,0,0,0])
# ranked, weight= Relief_configure(X,Y,K=1,prior='uniform', numUpdates='all', categoricalX='on')

# print('ranked')
# print(ranked)
# print('weight')
# print(weight)


In [ ]:
# def permute(x):
#   if len(x) == 7:
#     global X
#     global Y
#     Y.append((x[0] ^ x[1]))
#     X.append(x)
#     return
#   x = np.append(x,0)
#   permute(x)
#   x = x[0:-1]
#   x = np.append(x,1)
#   permute(x)

In [ ]:
# X = []
# Y = []
# x = np.array([], dtype=int)
# permute(x)
# X = np.array(X)
# Y = np.array(Y)
# ranked, weight=Relief_configure(X,Y, numUpdates='all', categoricalX='on')

# print('ranked')
# print(ranked)
# print('weight')
# print(weight)


In [ ]:
# # test classification dataset
# from sklearn.datasets import make_classification
# # define dataset
# X, Y = make_classification(n_samples=100, n_features=8, n_informative=3, n_redundant=0,shuffle=False, random_state=1)
# # summarize the dataset
# print(X.shape, Y.shape)
# ranked, weight= Relief_configure(X,Y,K=10,prior='uniform', numUpdates='all', categoricalX='off')

# print('ranked')
# print(ranked)
# print('weight')
# print(weight)